In [1]:
from src import Love_numbers_for_options_for_models_for_asymptotic_mu_ratios

anelasticity_description_ids = Love_numbers_for_options_for_models_for_asymptotic_mu_ratios(
    asymptotic_mu_ratios=[0.2],
    elasticity_model_names=["PREM"],
    long_term_anelasticity_model_names=["uniform"],
    short_term_anelasticity_model_names=["uniform"],
)

Description: PREM___uniform___uniform/asymptotic_mu_ratio__0.2:
----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: __short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: long_term_anelasticity____: Done.
----Run: __short_term_anelasticity__: Done.
----Run: long_term_anelasticity__short_term_anelasticity__: Done.


In [1]:
from src import  plot_Love_numbers_for_options_for_descriptions_per_type

plot_Love_numbers_for_options_for_descriptions_per_type(
    anelasticity_description_ids=[
        "PREM___uniform___uniform"
        + "/"
        + "asymptotic_mu_ratio__0.2",
    ],
    figsize=(10, 10), legend=False
)

In [1]:
from src import compute_anelastic_induced_harmonic_load_per_description_per_options

compute_anelastic_induced_harmonic_load_per_description_per_options(
    anelasticity_description_ids=[
        "PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
        "PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
    ],
)

Description: PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM/asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0:


----S: 100%|██████████| 89/89 [00:43<00:00,  2.03it/s]


----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.


----S: 100%|██████████| 89/89 [00:44<00:00,  2.00it/s]


----Run: __short_term_anelasticity__bounded_attenuation_functions: Done.


...

----Run: long_term_anelasticity____: Done.


----S: 100%|██████████| 89/89 [00:44<00:00,  1.99it/s]


----Run: __short_term_anelasticity__: Done.


----S: 100%|██████████| 89/89 [00:44<00:00,  1.99it/s]

----Run: long_term_anelasticity__short_term_anelasticity__: Done.


In [1]:
from src import plot_anelastic_induced_load_per_degree_per_description_per_options

plot_anelastic_induced_load_per_degree_per_description_per_options(
    anelasticity_description_ids=[
        "PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
        "PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
    ],
)

'\n    options=[\n        RunHyperParameters(\n            use_long_term_anelasticity=True, use_short_term_anelasticity=True, use_bounded_attenuation_functions=True\n        ), fontsize=20\n    ],\n)\n'

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [2]:
from src import plot_anelastic_induced_spatial_load_trend_per_description_per_options

plot_anelastic_induced_spatial_load_trend_per_description_per_options(
    anelasticity_description_ids=[
        "PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
        "PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
    ],num_colormesh_bins=15,
)

Description: PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM/asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0:
----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: __short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: long_term_anelasticity____: Done.
----Run: __short_term_anelasticity__: Done.
----Run: long_term_anelasticity__short_term_anelasticity__: Done.
Description: PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM/asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0:
----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: __short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: long_term_anelasticity____: Done.
----Run: __short_term_anelasticity__: Done.
----Run: long_term_anelasticity__short_term_anelasticity__: Done.
